# 4.9 Task Part 1

## 3. Import your analysis libraries, as well as your new customer data set as a dataframe.

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Defining path
path = r'C:\Users\Dudab\Oct 2023 - Instacart Analysis'

In [3]:
# Importing customers data set

customers = pd.read_csv (os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [4]:
# Checking imported data

customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
customers.shape

(206209, 10)

## 4. Wrangle the data so that it follows consistent logic; for example, rename columns with illogical names and drop columns that don’t add anything to your analysis.

In [6]:
# Looking at 'date_joined' column to see if all dates are the same

customers['date_joined'].value_counts(dropna = False)

date_joined
9/17/2018     213
2/10/2018     212
4/1/2019      211
9/21/2019     211
12/19/2017    210
             ... 
9/1/2018      141
1/22/2018     140
11/24/2017    139
7/18/2019     138
8/6/2018      128
Name: count, Length: 1187, dtype: int64

### Differences exist in the 'date_joined' field, so I won't treat this data as a "place-holder" and will maintain it in its current state.

In [7]:
# Renaming 'First Name' column to 'First_Name'

customers.rename(columns = {'First Name' : 'First_Name'}, inplace = True)

In [8]:
# Renaming 'Surnam' column to 'Last_Name'

customers.rename(columns = {'Surnam' : 'Last_Name'}, inplace = True)

In [9]:
# Renaming 'STATE' column to 'State'

customers.rename(columns = {'STATE' : 'State'}, inplace = True)

In [10]:
# Renaming 'n_dependants' column to 'number_of_dependants'

customers.rename(columns = {'n_dependants' : 'number_of_dependants'}, inplace = True)

In [11]:
# Checking output for column name changes

customers.head()

,user_id,First_Name,Last_Name,Gender,State,Age,date_joined,number_of_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# 5. Complete the fundamental data quality and consistency checks you’ve learned throughout this Achievement; for example, check for and address missing values and duplicates, and convert any mixed-type data.

In [12]:
# Using .describe() to look for inconsistencies

customers.describe()

,user_id,Age,number_of_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


### The 'user_id' doesn't necessarily have to be an integer, as it won't undergo mathematical calculations. Therefore, I will convert its data type to a string. The statistics for the remaining columns appear reasonable. However, it's somewhat surprising that the maximum number of dependents is only 3. One might expect some families to have more than 3 children or other dependents.

In [13]:
customers['user_id'] = customers['user_id'].astype('str')

In [14]:
# Checking the data type for 'user_id' has changed and checking data types for all other columns

customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   user_id               206209 non-null  object
 1   First_Name            194950 non-null  object
 2   Last_Name             206209 non-null  object
 3   Gender                206209 non-null  object
 4   State                 206209 non-null  object
 5   Age                   206209 non-null  int64 
 6   date_joined           206209 non-null  object
 7   number_of_dependants  206209 non-null  int64 
 8   fam_status            206209 non-null  object
 9   income                206209 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 15.7+ MB


### Given that the highest age for a client is 81 and the maximum number of dependents is 3, I'll optimize storage by converting both to int8 data types. Additionally, to conserve space, I'll switch the 'income' column to int32 for the same reasons.

In [15]:
customers['Age'] = customers['Age'].astype('int8')

In [16]:
customers['number_of_dependants'] = customers['number_of_dependants'].astype('int8')

In [17]:
customers['income'] = customers['income'].astype('int32')

In [18]:
# Checking data type changes

customers.dtypes

user_id                 object
First_Name              object
Last_Name               object
Gender                  object
State                   object
Age                       int8
date_joined             object
number_of_dependants      int8
fam_status              object
income                   int32
dtype: object

### Looking for mixed data types

In [19]:
for col in customers.columns.tolist():
    weird = (customers[[col]].map(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (customers[weird]) > 0:
        print (col)

First_Name


In [20]:
# The 'First_Name' column has mixed data types. These should be strings, so I will set the whole column to a string data type

customers['First_Name'] = customers['First_Name'].astype('str')

In [21]:
# Checking data type change

customers['First_Name'].dtype

dtype('O')

In [22]:
# Checking again for mixed data types

for col in customers.columns.tolist():
    weird = (customers[[col]].map(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (customers[weird]) > 0:
        print (col)

### There are no more mixed data type columns.

### Next step: I will look for missing values.

In [23]:
# Finding missing values and summing them

customers.isnull().sum()

user_id                 0
First_Name              0
Last_Name               0
Gender                  0
State                   0
Age                     0
date_joined             0
number_of_dependants    0
fam_status              0
income                  0
dtype: int64

### There are no missing values in any column.

### Next step: I will look for duplicate records.

In [24]:
# Looking for duplicate records

customers[customers.duplicated()]

,user_id,First_Name,Last_Name,Gender,State,Age,date_joined,number_of_dependants,fam_status,income


### There are no duplicate records in this data set.

In [25]:
# Making sure the data frame shape is the same after the consistency checks (since no records were excluded)

customers.shape

(206209, 10)

## 6. Combine your customer data with the rest of your prepared Instacart data. (Hint: Make sure the key columns are the same data type!)

In [26]:
# Importing data set to merge with customers data set.

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_updated_2.pkl'))

In [27]:
# Checking data set import

ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_spend,spender_flag,median_order_frequency,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,both,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,both,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,both,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,both,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,both,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [28]:
# Checking data type for 'user_id' column

ords_prods_merge['user_id'].dtype

dtype('int64')

In [29]:
# Changing the 'user_id' column to a string data type to match the data type for that same column in the customers data frame.

ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')

In [30]:
# Rechecking data type for 'user_id'

ords_prods_merge['user_id'].dtype

dtype('O')

### Before merging the 'ords_prods_merge' data frame with the 'customers' data frame, I will eliminate the '_merge' column. This column is now redundant, as the associated merge has been verified. Retaining it could interfere with the creation of a new indicator column for the upcoming merge.

In [31]:
# Dropping '_merge' column from ords_prods_merge data frame

ords_prods_merge = ords_prods_merge.drop(columns = ['_merge'])

In [32]:
# Using an inner join to merge the 'customers' data frame with the 'ords_prods_merge' data frame

ords_prods_custs_merge = ords_prods_merge.merge(customers, on = 'user_id', indicator = True)

In [33]:
# Checking final output

ords_prods_custs_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,First_Name,Last_Name,Gender,State,Age,date_joined,number_of_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [34]:
ords_prods_custs_merge.shape

(32404859, 32)

In [35]:
ords_prods_custs_merge['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [36]:
# Checking inner merge by looking at outer merge

merge_check = ords_prods_merge.merge(customers, on = 'user_id', indicator = True, how = 'outer')

In [37]:
merge_check['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

### It looks like all of the records match in both data frames.

In [38]:
ords_prods_custs_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_custs_merge.pkl'))